In [1]:
# load libraries
import os
%matplotlib inline
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',100)
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# load in datasets
# dataset from Kaggle
suicide_df = pd.read_csv("data/suicide_dataset.csv")
suicide_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2717 entries, 0 to 2716
Data columns (total 21 columns):
 #   Column                                                                                                             Non-Null Count  Dtype  
---  ------                                                                                                             --------------  -----  
 0   Country                                                                                                            2717 non-null   object 
 1   Sex                                                                                                                2196 non-null   object 
 2   Year                                                                                                               2717 non-null   int64  
 3   Suicide Rate                                                                                                       2196 non-null   float64
 4   Mental hospitals (per 10

In [5]:
suicide_df.head()

,Country,Sex,Year,Suicide Rate,Mental hospitals (per 100 000 population),Mental health units in general hospitals (per 100 000 population),Mental health outpatient facilities (per 100 000 population),Mental health day treatment facilities (per 100 000 population),Community residential facilities (per 100 000 population),Psychiatrists working in mental health sector (per 100 000 population),Nurses working in mental health sector (per 100 000 population),Social workers working in mental health sector (per 100 000 population),Psychologists working in mental health sector (per 100 000 population),Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),Beds in community residential facilities (per 100k population),Beds in mental hospitals (per 100k population),Beds for mental health in general hospitals (per 100k population)
0,Afghanistan,Both sexes,2000,8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Male,2000,14.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,Female,2000,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,Both sexes,2000,5.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Albania,Male,2000,8.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As can be seen, the original data put together from WHO and available in Kaggle has so many missing values.

In [21]:
missing = pd.concat([suicide_df.isnull().sum(), 100 * suicide_df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending = False)

,count,%
Beds in community residential facilities (per 100k population),2563,94.331984
Community residential facilities (per 100 000 population),2540,93.485462
Mental health day treatment facilities (per 100 000 population),2525,92.933382
Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),2505,92.197276
Social workers working in mental health sector (per 100 000 population),2461,90.577843
Year the law was enacted (latest revision),2440,89.804932
Psychologists working in mental health sector (per 100 000 population),2407,88.590357
Nurses working in mental health sector (per 100 000 population),2382,87.670225
Beds in mental hospitals (per 100k population),2370,87.228561
Mental hospitals (per 100 000 population),2366,87.081340


In [16]:
suicide_df[['Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%)','Year']].unique

AttributeError: 'DataFrame' object has no attribute 'unique'

In [17]:
# load other datasets to use as a dimension
sex_rt = pd.read_csv('data/suicide-death-rates-by-sex.csv')
age_rt = pd.read_csv('data/suicide-death-rate-by-age.csv')
inc_rt = pd.read_csv('data/suicide-rate-vs-income-inequality.csv') # gini index 
hom_rt = pd.read_csv('data/suicide-vs-homicide-rate.csv') 
sat_rt = pd.read_csv('data/suicide-rate-vs-self-reported-life-satisfaction.csv')
gun_rt = pd.read_csv('data/suicide-deaths-firearms.csv')
edu_rt = pd.read_csv('data/years-of-school.csv')
dep_rt = pd.read_csv('data/prevalence-of-depression.csv')
hap_tr = pd.read_csv('data/all_years.csv')
hr_rt = pd.read_csv('data/Human_Resources.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/suicide-rate-vs-prevalent-of-depression.csv'

In [7]:
inc_rt.head()

,Entity,Code,Year,"Suicide mortality rate (per 100,000 population)",GINI index (World Bank estimate),Total population (Gapminder),Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,2000,5.7,NaN,22856302.0,NaN
2,Afghanistan,AFG,2005,6.3,NaN,27614718.0,NaN
3,Afghanistan,AFG,2010,5.1,NaN,31411743.0,NaN
4,Afghanistan,AFG,2015,4.8,NaN,NaN,Asia


In [6]:
# rename the column names for faster manipulation
Nan_columns = suicide_df.iloc[:,4:22]
suicide_df.drop(Nan_columns, axis=1,inplace=True)
suicide_df.head(5)

,Country,Sex,Year,Suicide Rate
0,Afghanistan,Both sexes,2000,8.1
1,Afghanistan,Male,2000,14.3
2,Afghanistan,Female,2000,1.7
3,Albania,Both sexes,2000,5.8
4,Albania,Male,2000,8.2


In [ ]:
# load supplemental datasets